# Obtaining the Postal Codes, Boroughs, and Neighborhoods of Toronto

## Import Libraries needed for assignment

In [1]:
import os.path

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

from bs4 import BeautifulSoup
import re

print('Libraries imported.')

Libraries imported.


## Decided to save webpage to an html file in the event the wiki page gets editted incorrectly by someone.

If the html file exists, then the file is read into `wiki_html`

If the html file does not exist, then the webpage is scraped(in its current version) to `wiki_html`. Which then gets saved to a html file.

Finally `wiki_html` is used to create BeautifulSoup object.

In [2]:
if os.path.exists('wiki.html'):   
    with open('wiki.html','rb') as html:
        wiki_html = html.read()
else:
    wiki_page = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
    wiki_response = requests.get(wiki_page)
    wiki_html = wiki_response.content
    with open('wiki.html','wb') as html: #writes as binary file
        html.write(wiki_html)

wiki_soup = BeautifulSoup(wiki_html, 'html.parser')


## Using the BeautifulSoup object (`wiki_soup`) to create pandas dataframe with columns of PostalCode, Borough, Neighborhood

First, the table with the required data is located. 

For each assigned Postal Code:

The data (Postal Code, Borough, Neighborhood) for each assigned Postal Code is put into a dictionary (`cell`).

The `cell['Borough']` and `cell['Neighborhood']` lines grabs all the Boroughs and Neighborhoods for the current Postal Code.

Then the `cell` dictionary is appended to a list which will be converted into a pandas dataframe.

Final step is too replace some of the Boroughs with easier to read names


In [3]:
wiki_table_contents = []
wiki_table = wiki_soup.find('table') # Locates table with required data

for row in wiki_table.findAll('td'): # Goes through each of the table's cells and extracts the data associated with each assigned Postal Code
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = "/ ".join(elem for elem in re.findall(r"(.*?)(?:\(.*?\)|$)", row.span.text)).replace('/ ','/').strip('/ ')         # (row.span.text).split('(')[0]
        cell['Neighborhood'] = ",".join([elem for elem in re.findall('\((.*?)\)', row.span.text)]).replace(' / ',',').strip(' ')  # (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        wiki_table_contents.append(cell)
        
df=pd.DataFrame(wiki_table_contents) 
df['Borough'] = df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})
df

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park,Harbourfront"
3,M6A,North York,"Lawrence Manor,Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Malvern,Rouge"
7,M3B,North York/North,Don Mills
8,M4B,East York,"Parkview Hill,Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [4]:
print('The shape of this dataframe is: ', df.shape)

The shape of this dataframe is:  (103, 3)


# Obtaining Coordinates for Toronto's neighborhoods

Using the provided Geospatial_Coordinates.csv, the csv is read into the `coords_df` dataframe.

The `Postal Code` column is renamed to `PostalCode` in order to merge with the `df` dataframe to create the new combined dataframe : `postalCode_df'



In [5]:
coords_df = pd.read_csv('Geospatial_Coordinates.csv').rename(columns={'Postal Code': 'PostalCode'})

In [6]:
postalCode_df = pd.merge(df,coords_df, on='PostalCode')
postalCode_df

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park,Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor,Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Malvern,Rouge",43.806686,-79.194353
7,M3B,North York/North,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill,Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


# Clustering the **Toronto** neighboorhoods

## First, find all neighboorhoods within all the boroughs with `Toronto` in their name. 

The `Toronto` dataframe contains all the neighborhoods with `Toronto` in the borough name.


In [7]:
Toronto = postalCode_df[postalCode_df['Borough'].str.contains('Toronto')].reset_index(drop=True)
Toronto

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park,Harbourfront",43.654260,-79.360636
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
3,M4E,East Toronto,The Beaches,43.676357,-79.293031
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
5,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
6,M6G,Downtown Toronto,Christie,43.669542,-79.422564
7,M5H,Downtown Toronto,"Richmond,Adelaide,King",43.650571,-79.384568
8,M6H,West Toronto,"Dufferin,Dovercourt Village",43.669005,-79.442259
9,M4J,East York/East Toronto,The Danforth East,43.685347,-79.338106


In [8]:
print("The shape of the Toronto dataframe is: ", Toronto.shape)

The shape of the Toronto dataframe is:  (39, 5)


## Next, grabbing the coordinates of Toronto

In [9]:
address = 'Toronto, ON'
geolocator = Nominatim(user_agent="on_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


## Let's visualize the neighboorhoods of `Toronto`

In [11]:
# create map of Downtone Toronto using latitude and longitude values
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, label in zip(Toronto['Latitude'], Toronto['Longitude'], Toronto['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto